In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
from sklearn.linear_model import LinearRegression

### Создаем искусственный сэмпл 

Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. 
По образу практики, попробуйте создать искусственный датасет с лишними столбцами. 
Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. 
Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. 
Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. 
Данное задание не имеет какого-то “правильного” решения. 
Цель - достичь базового понимания проблемы. 
Чем больше вы фантазируете, тем лучше :) 
Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

#### создадим датасет пациентов и параметров их здоровья. 
В качестве целевой переменной будем использовать верхнее артериальное давление.
Создадим функцию определяющую верхнее артериальное давление в зависимости от пола (мужской пол более предрасположен к повышенному АД), роста и веса (по индексу массы тела = вес/рост в м**2), а также возраста (чем человек старше, тем выше давление)
При этом частота дыхания в генерируемом датасете есть, но по частоте дыхания верхнее АД не зависит (наше допущение)

In [2]:
n_samples = 1000 #кол-во примеров в датасете
weight = list(range(50,151,1))
growth = list(range(140,201,1))
breathing_rate = list(range(10,31,1))
age = list(range(18,81,1))
age_patient = np.random.choice(age, n_samples) + 18 #возраст пациента
weight_patient = np.random.choice(weight, n_samples) + 30 #вес пациента
growth_patient = np.random.choice(growth, n_samples) + 10 #рост пациента
breathing_rate = np.random.choice(breathing_rate, n_samples, p=[0.02, 0.022, 0.025, 0.05, 0.1, 0.125, 0.15, 0.125, 0.1,0.075,0.05, 0.05,0.025, 0.025, 0.01,0.01,0.01,0.01,0.01,0.004,0.004]) #частота дыхания
sex_patient = np.random.choice([0,1], n_samples, p=[0.5, 0.5]) #пол пациента, 1 - мужской, 0 - женский

upper_pressure = weight_patient/((growth_patient/100)**2) + age_patient/5 + sex_patient*5 + 80 #прописываем ф-лу зависимости верхнего артериального давления
#генерируем датасет 
data = pd.DataFrame({'age_patient': age_patient, 'weight_patient': weight_patient, 'growth_patient': growth_patient, 'breathing_rate': breathing_rate, 'sex_patient': sex_patient,'upper_pressure': upper_pressure})
data.head(10)

,age_patient,weight_patient,growth_patient,breathing_rate,sex_patient,upper_pressure
0,70,142,163,14,0,147.445745
1,47,108,180,14,0,122.733333
2,69,99,194,11,1,125.104602
3,55,139,174,15,1,141.910953
4,92,109,201,17,0,125.379530
5,45,171,158,15,1,162.498638
6,65,119,207,17,0,120.771943
7,92,157,189,15,1,147.351737
8,71,94,210,16,1,120.515193
9,37,87,155,12,0,123.612279


#### На основании полученного датасета попробуем спрогнозировать целевую переменную upper_pressure, используя линейную регрессию и набор всех параметров ['age_patient', 'weight_patient', 'growth_patient','breathing_rate', 'sex_patient']

In [3]:
from sklearn.metrics import mean_absolute_error

X = data[['age_patient', 'weight_patient', 'growth_patient','breathing_rate', 'sex_patient']]
y = data['upper_pressure']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['age_patient', 'weight_patient', 'growth_patient','breathing_rate', 'sex_patient']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 0.20162032  0.3181277  -0.4662078   0.01947924  4.99931016]
Bias: 163.4399695838024
Error: 1.6847074028438702


Мы видим, что ошибка не такая большая, но все же есть, при этом bias переоценен, попробуем в следующем шаге убрать breathing_rate

In [4]:
X1 = data[['age_patient', 'weight_patient', 'growth_patient', 'sex_patient']]
y1 = data['upper_pressure']
reg1 = LinearRegression().fit(X1, y1)
print('Weights: {}'.format(reg1.coef_))
print('Bias: {}'.format(reg1.intercept_))

pred_values1 = reg1.predict(data[['age_patient', 'weight_patient', 'growth_patient', 'sex_patient']])
print('Error: {}'.format(mean_absolute_error(pred_values1, y1)))

Weights: [ 0.20158747  0.3180256  -0.46626363  5.00307343]
Bias: 163.80004534092853
Error: 1.6854509990410953


Bias почти не снизился, ошибка на том же уровне, пробуем создать новый признак. Т.к. для weight_patient вес > 0, а для growth_patient < 0 - пробуем разделить weight_patient/growth_patient (у growth_patient отрицательная зависимость)

In [5]:
# Создаем новый признак
data['growth_patient-1'] = 1 / data['growth_patient'] # т.к. по коэф зависимость сильная но отрицательная
data['mult'] = data['weight_patient'] * data['growth_patient-1'] # Создаем новый признак
data.head(5)

,age_patient,weight_patient,growth_patient,breathing_rate,sex_patient,upper_pressure,growth_patient-1,mult
0,70,142,163,14,0,147.445745,0.006135,0.871166
1,47,108,180,14,0,122.733333,0.005556,0.600000
2,69,99,194,11,1,125.104602,0.005155,0.510309
3,55,139,174,15,1,141.910953,0.005747,0.798851
4,92,109,201,17,0,125.379530,0.004975,0.542289


In [27]:
X2 = data[['age_patient', 'weight_patient', 'growth_patient', 'sex_patient','mult']]
y2 = data['upper_pressure']
reg2 = LinearRegression().fit(X2, y2)
print('Weights: {}'.format(reg2.coef_))
print('Bias: {}'.format(reg2.intercept_))

pred_values2 = reg2.predict(data[['age_patient', 'weight_patient', 'growth_patient', 'sex_patient','mult']])
print('Error: {}'.format(mean_absolute_error(pred_values2, y2)))

Weights: [ 1.99233291e-01 -3.62901100e-01  3.34099842e-02  4.99766229e+00
  1.21369225e+02]
Bias: 74.01234736460945
Error: 0.27465276737532757


Ошибка значительно снизилась, bias значительно снизился.

Но мы можем добавить мощности оригинальных функций в качестве новых, сгенериров уравнение еще более высокого порядка

In [9]:
from sklearn.preprocessing import PolynomialFeatures

In [10]:
poly = PolynomialFeatures(2)

In [19]:
X_p = poly.fit_transform(data[['age_patient', 'weight_patient','breathing_rate', 'sex_patient', 'growth_patient-1']])

In [20]:
poly.get_feature_names()

['1',
 'x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x0^2',
 'x0 x1',
 'x0 x2',
 'x0 x3',
 'x0 x4',
 'x1^2',
 'x1 x2',
 'x1 x3',
 'x1 x4',
 'x2^2',
 'x2 x3',
 'x2 x4',
 'x3^2',
 'x3 x4',
 'x4^2']

In [21]:
reg = LinearRegression().fit(X_p,y)

In [22]:
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(X_p)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 0.00000000e+00  2.00897309e-01 -3.17870121e-01 -1.57949503e-02
  2.53445863e+00 -1.47579327e+04 -1.44777456e-06 -6.55470073e-06
  2.68143440e-06 -7.42363123e-04  8.13684201e-02  6.53902915e-07
 -1.01889490e-05 -1.37491959e-04  1.13432583e+02  2.11021053e-04
 -2.20152972e-04  1.77939235e+00  2.53449072e+00 -4.31304412e-01
  1.29820317e+06]
Bias: 121.46416824586963
Error: 0.056840373506231556


 * x0 = 2.00897309e-01 age_patient
 * x1= -3.17870121e-01 weight_patient
 * x3 = 2.53445863e+00 sex_patient
 * x4 = -1.47579327e+04 growth_patient-1
 * x1 x4 1.13432583e+02
 * x2 x4 1.77939235e+00 #выкидываем т.к. выкинули x2
 * x3^2 2.53449072e+00
 * x4^2 1.29820317e+06
 только эти коэф вносят сущест вклад в модель (это как раз частота дыхания)
 
 x2 = -1.57949503e-02 - существенный вклад не вносит breathing_rate

In [25]:
# Создаем новые признаки, которые вносят существенный вклад в модель
data['mult_1'] = data['weight_patient'] * data['growth_patient-1'] #x1 x4 1.13432583e+02
data['mult_2'] = data['sex_patient'] * data['sex_patient'] #x3^2 2.53449072e+00
data['mult_3'] = data['growth_patient-1'] * data['growth_patient-1'] #x4^2 1.29820317e+06
data.head(5)

,age_patient,weight_patient,growth_patient,breathing_rate,sex_patient,upper_pressure,growth_patient-1,mult,mult_1,mult_2,mult_3
0,70,142,163,14,0,147.445745,0.006135,0.871166,0.871166,0,0.000038
1,47,108,180,14,0,122.733333,0.005556,0.600000,0.600000,0,0.000031
2,69,99,194,11,1,125.104602,0.005155,0.510309,0.510309,1,0.000027
3,55,139,174,15,1,141.910953,0.005747,0.798851,0.798851,1,0.000033
4,92,109,201,17,0,125.379530,0.004975,0.542289,0.542289,0,0.000025


In [26]:
#Предсказываем значение и смотрим ошибку на выбранных нами переменных
X_poly2 = data[['age_patient', 'weight_patient', 'growth_patient-1', 'sex_patient','mult_1','mult_2','mult_3']]
y_poly2 = data['upper_pressure']
reg_poly2 = LinearRegression().fit(X_poly2, y_poly2)
print('Weights: {}'.format(reg_poly2.coef_))
print('Bias: {}'.format(reg_poly2.intercept_))

pred_values_poly2 = reg_poly2.predict(data[['age_patient', 'weight_patient', 'growth_patient-1', 'sex_patient','mult_1','mult_2','mult_3']])
print('Error: {}'.format(mean_absolute_error(pred_values_poly2, y_poly2)))

Weights: [ 2.00018569e-01 -3.18510317e-01 -1.47416930e+04  2.49814830e+00
  1.13460129e+02  2.49812791e+00  1.29964768e+06]
Bias: 121.3720044385644
Error: 0.05709249874225985


#### Наименьшую ошибку дала последняя модель reg_poly2
* reg ('age_patient', 'weight_patient', 'growth_patient', 'sex_patient','mult') Bias: 163.4399695838024 Error: 1.6847074028438702
* reg1 ('age_patient', 'weight_patient', 'growth_patient', 'sex_patient') Bias: 163.80004534092853 Error: 1.6854509990410953
* reg2 ('age_patient', 'weight_patient', 'growth_patient', 'sex_patient','mult') Bias: 74.01234736460945 Error: 0.27465276737532757
* reg_poly2 ('age_patient', 'weight_patient', 'growth_patient-1', 'sex_patient','mult_1','mult_2','mult_3') Bias: 121.3720044385644 Error: 0.05709249874225985